In [ ]:
import pandas as pd
import datetime 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

df = pd.read_excel('../../DataSources/bellingcat_all_inclu_war_lang_thread.xlsx')
df['date'] = pd.to_datetime(df['date']).dt.date
df = df[df['date']<datetime.date(year=2022,month=6,day=1)]
df['date'] = pd.to_datetime(df['date'])


print(df.head())



In [ ]:
df['type_of_tweet']=df['type_of_tweet'].replace(['Bellingcat_Thread', 'Normal Tweet',"quoted"], 'Conversation')

In [ ]:
df_followers= pd.read_excel("..\..\DataSources\Followers_final.xlsx")
df_followers = df_followers[["Date","followers"]].groupby([pd.Grouper(freq="M",key="Date")])["followers"].max().reset_index()

In [ ]:
df_followers['year'] = df_followers.Date.dt.year
df_followers['month'] = df_followers.Date.dt.month
print(df_followers.head())

In [ ]:
def f(x):
    return pd.Series(dict(retweets =  x['retweets'].sum(),
                        likes = x['likes'].sum(),
                        replys = x['replys'].sum(), 
                        quotes =  x['quotes'].sum(),
                        count =  x['text'].count(),
                        lang = x["lang"].mode().iat[0], 
                        date = x['date'].max(), 
                        ))

        
    
#df_bellingcat_totals = df.groupby([pd.Grouper(freq="M",key="date"),"type_of_tweet"]).apply(f).reset_index()      
df_bellingcat_totals = df.groupby(["conversation_id","type_of_tweet"]).apply(f).reset_index()

In [ ]:
df_bellingcat_totals =df_bellingcat_totals[df_bellingcat_totals['lang']=="en"]

In [ ]:
df_bellingcat_totals["count"] =np.where(df_bellingcat_totals["type_of_tweet"] =="Conversation",1,df_bellingcat_totals["count"])

In [ ]:
def g(x):
    return pd.Series(dict(retweets =  x['retweets'].sum(),
                        likes = x['likes'].sum(),
                        replys = x['replys'].sum(), 
                        quotes =  x['quotes'].sum(),
                        count =  x['count'].sum(),
                        lang = x["lang"].mode().iat[0]
                        ))


df_bellingcat_totals = df_bellingcat_totals.groupby([pd.Grouper(freq="M",key="date"),"type_of_tweet"]).apply(g).reset_index()      

In [ ]:
df_bellingcat_totals['year'] = df_bellingcat_totals.date.dt.year
df_bellingcat_totals['month'] = df_bellingcat_totals.date.dt.month


In [ ]:
print(df_bellingcat_totals.head())

In [ ]:
df_bellingcat_totals=df_bellingcat_totals.merge(df_followers, on=["year","month"], how='left').drop(columns=['month', 'year', 'Date'])

In [ ]:
#df_bellingcat_totals.to_excel("text.xlsx")

In [ ]:
print(df_bellingcat_totals.head(20))

In [ ]:
df_bellingcat_totals["Monthly_retweets"] = df_bellingcat_totals["retweets"]/df_bellingcat_totals["count"]
df_bellingcat_totals["Monthly_replies"] = df_bellingcat_totals["replys"]/df_bellingcat_totals["count"]
df_bellingcat_totals["Monthly_likes"] = df_bellingcat_totals["likes"]/df_bellingcat_totals["count"]
df_bellingcat_totals["Monthly_quotes"] = df_bellingcat_totals["quotes"]/df_bellingcat_totals["count"]

In [ ]:
print(df_bellingcat_totals.tail())

In [ ]:
print(df_bellingcat_totals.head())

In [ ]:
df_bellingcat_totals["all_interactions"] = df_bellingcat_totals["quotes"]+df_bellingcat_totals["retweets"]+df_bellingcat_totals["likes"]+df_bellingcat_totals["replys"]

In [ ]:
def h(x):
    return pd.Series(dict(all_interactions =  x['all_interactions'].sum(),
                        followers = x["followers"].max()
                        ))


df_bellingcat_totals_gruped = df_bellingcat_totals.groupby([pd.Grouper(freq="M",key="date")]).apply(h).reset_index() 



In [ ]:
#df_bellingcat_totals_gruped.to_excel("text.xlsx")

In [ ]:
df_bellingcat_totals_gruped

In [ ]:
df_bellingcat_totals_gruped_timeIndex = df_bellingcat_totals_gruped.set_index('date')
percentage_change = df_bellingcat_totals_gruped_timeIndex.pct_change()
percentage_change.to_excel("text.xlsx")

In [ ]:
df_bellingcat_totals_gruped.to_csv("..\..\DataSources\Dataset_Graphs\RQ1_Followers_Interactions_Tweets\Followers_Interactions.csv")

In [ ]:
#df_bellingcat_totals.to_csv("trends_bellingcat_total.csv")

In [ ]:


df_bellingcat_year_type =    df_bellingcat_totals.groupby([pd.Grouper(freq="Y",key="date"),"type_of_tweet"])["count"].sum().reset_index()

df_bellingcat_year_type["relative"] = df_bellingcat_year_type["count"]/df_bellingcat_year_type.groupby([pd.Grouper(freq="M",key="date")])["count"].transform("sum")

In [ ]:
df_bellingcat_year_type

In [ ]:
df_bellingcat_year_type.to_excel("relative.xlsx")

In [ ]:

df_bellingcat_year_type.to_csv("..\..\DataSources\Dataset_Graphs\RQ1_Followers_Interactions_Tweets\Relative_Tweets.csv")

In [ ]:
df_bellingcat_year =    df_bellingcat_totals.groupby(["type_of_tweet"])["count"].sum().reset_index()

In [ ]:
df_bellingcat_year